In [22]:
pwd

'/content/drive/My Drive/MRV'

In [4]:
cd drive/MyDrive/

/content/drive/MyDrive


In [6]:
cd MRV

/content/drive/MyDrive/MRV


In [7]:
ls

architecture.py  encodings/  Faces/  model/


In [9]:
!pip install -q mtcnn

     |████████████████████████████████| 2.3MB 3.0MB/s 


In [10]:
from architecture import *
import os
import cv2
import mtcnn
import pickle
import numpy as np
from sklearn.preprocessing import Normalizer
from tensorflow.keras.models import load_model

In [23]:
face_data = "Faces/"
required_shape = (160, 160)
face_encoder = InceptionResNetV2()
path = "model/facenet_keras_weights.h5"
face_encoder.load_weights(path)
face_detector = mtcnn.MTCNN()
encodes = []
encoding_dict = dict()
l2_normalizer = Normalizer("l2")

## Training Code

In [24]:
def normalize(img):
    mean, std = img.mean(), img.std()
    return (img - mean) / std

In [25]:
for face_names in os.listdir(face_data):
    person_dir = os.path.join(face_data, face_names)

    for image_name in os.listdir(person_dir):
        image_path = os.path.join(person_dir, image_name)

        img_BGR = cv2.imread(image_path)
        img_RGB = cv2.cvtColor(img_BGR, cv2.COLOR_BGR2RGB)

        x = face_detector.detect_faces(img_RGB)
        x1, y1, width, height = x[0]["box"]
        x1, y1 = abs(x1), abs(y1)
        x2, y2 = x1 + width, y1 + height
        face = img_RGB[y1:y2, x1:x2]

        face = normalize(face)
        face = cv2.resize(face, required_shape)
        face_d = np.expand_dims(face, axis=0)
        encode = face_encoder.predict(face_d)[0]
        encodes.append(encode)

    if encodes:
        encode = np.sum(encodes, axis=0)
        encode = l2_normalizer.transform(np.expand_dims(encode, axis=0))[0]
        encoding_dict[face_names] = encode

In [26]:
path = "encodings/encodings.pkl"
with open(path, "wb") as file:
    pickle.dump(encoding_dict, file)

## Detection Code

In [ ]:
import cv2
import numpy as np
import mtcnn
from architecture import *
from train import normalize, l2_normalizer
from scipy.spatial.distance import cosine
from tensorflow.keras.models import load_model
import pickle

In [ ]:
confidence_t = 0.99
recognition_t = 0.5
required_size = (160, 160)

In [ ]:
def get_face(img, box):
    x1, y1, width, height = box
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    face = img[y1:y2, x1:x2]
    return face, (x1, y1), (x2, y2)

In [ ]:
def get_encode(face_encoder, face, size):
    face = normalize(face)
    face = cv2.resize(face, size)
    encode = face_encoder.predict(np.expand_dims(face, axis=0))[0]
    return encode

In [ ]:
def load_pickle(path):
    with open(path, "rb") as f:
        encoding_dict = pickle.load(f)
    return encoding_dict

In [ ]:
def detect(img, detector, encoder, encoding_dict):
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = detector.detect_faces(img_rgb)
    for res in results:
        if res["confidence"] < confidence_t:
            continue
        face, pt_1, pt_2 = get_face(img_rgb, res["box"])
        encode = get_encode(encoder, face, required_size)
        encode = l2_normalizer.transform(encode.reshape(1, -1))[0]
        name = "unknown"

        distance = float("inf")
        for db_name, db_encode in encoding_dict.items():
            dist = cosine(db_encode, encode)
            if dist < recognition_t and dist < distance:
                name = db_name
                distance = dist

        if name == "unknown":
            cv2.rectangle(img, pt_1, pt_2, (0, 0, 255), 2)
            cv2.putText(img, name, pt_1, cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 1)
        else:
            cv2.rectangle(img, pt_1, pt_2, (0, 255, 0), 2)
            cv2.putText(
                img,
                name + f"__{distance:.2f}",
                (pt_1[0], pt_1[1] - 5),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (0, 200, 200),
                2,
            )
    return img

In [ ]:
if __name__ == "__main__":
    required_shape = (160, 160)
    face_encoder = InceptionResNetV2()
    path_m = "model/facenet_keras_weights.h5"
    face_encoder.load_weights(path_m)
    encodings_path = "encodings/encodings.pkl"
    face_detector = mtcnn.MTCNN()
    encoding_dict = load_pickle(encodings_path)

    cap = cv2.VideoCapture(0)

    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            print("CAM NOT OPEN")
            break

        frame = detect(frame, face_detector, face_encoder, encoding_dict)

        cv2.imshow("camera", frame)

        if cv2.waitKey(1) & 0xFF == ord("q"):
            break